In [ ]:
import numpy as np
import pandas as pd

We will load the data now:

*   Consolidées et définitives 2013-2022:
https://odre.opendatasoft.com/explore/dataset/eco2mix-national-cons-def/information/?disjunctive.nature
*   Au pas quart d'heure 2022-2023:
https://odre.opendatasoft.com/explore/dataset/eco2mix-national-tr/information/?disjunctive.nature
*   European wholesale prices:
https://ember-climate.org/data-catalogue/european-wholesale-electricity-price-data/

In [ ]:
# Transform MW quarterly hour to consolidated MWh per hour
# load data
from google.colab import drive
drive.mount('/content/gdrive')
df=pd.read_csv('gdrive/My Drive/Data_CFE/eco2mix-national-cons-def.csv', sep=";")
df=df.sort_values(by="Date et Heure")

# Drop columns by name
columns_to_drop = ['Ech. physiques (MW)', 'Ech. comm. Angleterre (MW)', 'Ech. comm. Espagne (MW)', 'Ech. comm. Italie (MW)', 'Ech. comm. Suisse (MW)', 'Ech. comm. Allemagne-Belgique (MW)',
                   'Gaz - Cogénération (MW)','Fioul - Autres (MW)','Gaz - TAC (MW)', 'Gaz - CCG (MW)', 'Gaz - Autres (MW)',
                   'Taux de CO2 (g/kWh)', 'Fioul - TAC (MW)', 'Fioul - Cogénération (MW)', 'Hydraulique - Lacs (MW)',
                   "Hydraulique - Fil de l'eau + éclusée (MW)", 'Hydraulique - STEP turbinage (MW)', 'Bioénergies - Déchets (MW)', 'Bioénergies - Biomasse (MW)', 'Bioénergies - Biogaz (MW)',
                   'Pompage (MW)', 'Prévision J-1 (MW)', 'Prévision J (MW)' ]
df = df.drop(columns=columns_to_drop)
to_keep = ["00:00", "01:00", "02:00", "03:00", "04:00", "05:00", "06:00", "07:00", "08:00", "09:00", "10:00", "11:00", "12:00", "13:00", "14:00", "15:00", "16:00", "17:00", "18:00", "19:00", "20:00", "21:00", "22:00", "23:00",
           "00:30", "01:30", "02:30", "03:30", "04:30", "05:30", "06:30", "07:30", "08:30", "09:30", "10:30", "11:30", "12:30", "13:30", "14:30", "15:30", "16:30", "17:30", "18:30", "19:30", "20:30", "21:30", "22:30", "23:30"]
df = df[df["Heure"].isin(to_keep)]

df = df.reset_index(drop=True)
df = df.drop(df.index[0])
df = df.reset_index(drop=True)

df2=df

# Convert 'Heure' column to datetime
df2['Heure_bis'] = pd.to_datetime(df2['Heure'], format='%H:%M')

# Group by 'Date' and 'Hour', then sum the 'Consommation (MW)' values
hourly_df = df2.groupby(['Date', df['Heure_bis'].dt.hour])['Consommation (MW)', 'Fioul (MW)', 'Charbon (MW)', 'Gaz (MW)', 'Nucléaire (MW)', 'Eolien (MW)', 'Solaire (MW)', 'Hydraulique (MW)', 'Bioénergies (MW)'].sum().reset_index()

# Convert 'Heure_bis' to the format '00:00'
hourly_df['Heure_bis'] = hourly_df['Heure_bis'].astype(str).str.zfill(2) + ":00"
# Rename the 'Heure_bis' column to 'Heure'
hourly_df.rename(columns={'Heure_bis': 'Heure'}, inplace=True)

#hourly_df['Consommation (MW)'] = df['Consommation (MW)'] / 2
columns_to_divide_by_2=['Consommation (MW)', 'Fioul (MW)', 'Charbon (MW)', 'Gaz (MW)', 'Nucléaire (MW)', 'Eolien (MW)', 'Solaire (MW)', 'Hydraulique (MW)', 'Bioénergies (MW)']
hourly_df[columns_to_divide_by_2]=hourly_df[columns_to_divide_by_2]/2

df= hourly_df

df = df.reset_index(drop=True)
df = df.drop(df.index[0])
df = df.reset_index(drop=True)

# Combine 'Date' and 'Heure' into a new column 'Datetime'
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Heure'])


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


<ipython-input-82-b3690c177327>:5: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('gdrive/My Drive/Data_CFE/eco2mix-national-cons-def.csv', sep=";")
<ipython-input-82-b3690c177327>:29: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  hourly_df = df2.groupby(['Date', df['Heure_bis'].dt.hour])['Consommation (MW)', 'Fioul (MW)', 'Charbon (MW)', 'Gaz (MW)', 'Nucléaire (MW)', 'Eolien (MW)', 'Solaire (MW)', 'Hydraulique (MW)', 'Bioénergies (MW)'].sum().reset_index()


In [ ]:
# load df_target

df_target=pd.read_csv('gdrive/My Drive/Data_CFE/France.csv')
df_target=df_target.sort_values(by="Datetime (Local)")

# We merge now the two datasets:

# Convert the 'Datetime (Local)' column to datetime format in df_target
df_target['Datetime (Local)'] = pd.to_datetime(df_target['Datetime (Local)'])

# Create ID in both dataframes
df['ID'] = df['Datetime']
df_target['ID'] = df_target['Datetime (Local)']

# Join df and df_target on ID
result = pd.merge(df, df_target, on='ID', how='inner')

# We drop some recurrent variables
to_drop = ['ISO3 Code', 'ID', 'Datetime (UTC)'] #'Date', 'Heure']
result = result.drop(columns=to_drop)

In [ ]:
result.tail(5)

,Date,Heure,Consommation (MW),Fioul (MW),Charbon (MW),Gaz (MW),Nucléaire (MW),Eolien (MW),Solaire (MW),Hydraulique (MW),Bioénergies (MW),Datetime,Country,Datetime (Local),Price (EUR/MWhe)
64985,2022-05-31,19:00,48218.5,83.0,522.5,4031.5,30522.5,2631.0,2692.5,8493.5,1219.0,2022-05-31 19:00:00,France,2022-05-31 19:00:00,249.80
64986,2022-05-31,20:00,46607.5,83.5,555.0,4075.0,30511.0,2208.0,992.0,9260.0,1219.0,2022-05-31 20:00:00,France,2022-05-31 20:00:00,254.67
64987,2022-05-31,21:00,44360.0,82.5,525.0,3977.5,30495.0,1709.5,149.0,8827.5,1220.5,2022-05-31 21:00:00,France,2022-05-31 21:00:00,247.51
64988,2022-05-31,22:00,44543.0,82.5,233.0,3893.5,30444.0,1788.0,5.5,8214.0,1216.0,2022-05-31 22:00:00,France,2022-05-31 22:00:00,240.01
64989,2022-05-31,23:00,45991.0,81.5,2.5,3840.5,30404.5,2060.5,6.0,7925.5,1222.0,2022-05-31 23:00:00,France,2022-05-31 23:00:00,227.98


In [ ]:
print(result.iloc[63606]) #max 2987,78 on 4 April 2022 iloc=63606

Date                          2022-04-04
Heure                              08:00
Consommation (MW)                73257.0
Fioul (MW)                        1493.0
Charbon (MW)                       465.5
Gaz (MW)                          8495.5
Nucléaire (MW)                   32110.0
Eolien (MW)                       4408.5
Solaire (MW)                       720.0
Hydraulique (MW)                 16159.5
Bioénergies (MW)                  1019.5
Datetime             2022-04-04 08:00:00
Country                           France
Datetime (Local)     2022-04-04 08:00:00
Price (EUR/MWhe)                 2987.78
Name: 63606, dtype: object


In [ ]:
# We reduce the shape of our data set for time computations limits
df = result

df['Series'] = 0
df["IndexTime"] = df.index

In [ ]:
# Save to CSV files
df.to_csv('gdrive/My Drive/Data_CFE/df.csv', index=False)

Testing the coherence of the dataset:

In [ ]:
test=df

# Transform MW quarterly hour to consolidated MWh per hour
# load data
from google.colab import drive
drive.mount('/content/gdrive')
df=pd.read_csv('gdrive/My Drive/Data_CFE/eco2mix-national-cons-def.csv', sep=";")
df=df.sort_values(by="Date et Heure")

# Drop columns by name
columns_to_drop = ['Ech. physiques (MW)', 'Ech. comm. Angleterre (MW)', 'Ech. comm. Espagne (MW)', 'Ech. comm. Italie (MW)', 'Ech. comm. Suisse (MW)', 'Ech. comm. Allemagne-Belgique (MW)',
                   'Gaz - Cogénération (MW)','Fioul - Autres (MW)','Gaz - TAC (MW)', 'Gaz - CCG (MW)', 'Gaz - Autres (MW)',
                   'Taux de CO2 (g/kWh)', 'Fioul - TAC (MW)', 'Fioul - Cogénération (MW)', 'Hydraulique - Lacs (MW)',
                   "Hydraulique - Fil de l'eau + éclusée (MW)", 'Hydraulique - STEP turbinage (MW)', 'Bioénergies - Déchets (MW)', 'Bioénergies - Biomasse (MW)', 'Bioénergies - Biogaz (MW)',
                   'Pompage (MW)', 'Prévision J-1 (MW)', 'Prévision J (MW)' ]
df = df.drop(columns=columns_to_drop)
to_keep = ["00:00", "01:00", "02:00", "03:00", "04:00", "05:00", "06:00", "07:00", "08:00", "09:00", "10:00", "11:00", "12:00", "13:00", "14:00", "15:00", "16:00", "17:00", "18:00", "19:00", "20:00", "21:00", "22:00", "23:00",
           "00:30", "01:30", "02:30", "03:30", "04:30", "05:30", "06:30", "07:30", "08:30", "09:30", "10:30", "11:30", "12:30", "13:30", "14:30", "15:30", "16:30", "17:30", "18:30", "19:30", "20:30", "21:30", "22:30", "23:30"]
df = df[df["Heure"].isin(to_keep)]

df = df.reset_index(drop=True)
df = df.drop(df.index[0])
df = df.reset_index(drop=True)



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


<ipython-input-89-eac9ad0dfbf3>:7: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('gdrive/My Drive/Data_CFE/eco2mix-national-cons-def.csv', sep=";")


In [ ]:
df3 = df[df['Date'] == '2016-08-26']
df3.head(50)

,Périmètre,Nature,Date,Heure,Date et Heure,Consommation (MW),Fioul (MW),Charbon (MW),Gaz (MW),Nucléaire (MW),Eolien (MW),Solaire (MW),Hydraulique (MW),Bioénergies (MW)
28943,France,Données définitives,2016-08-26,00:00,2016-08-26T00:00:00+02:00,48656.0,118.0,251.0,3369.0,38591.0,1010.0,-1.0,5955.0,962.0
28944,France,Données définitives,2016-08-26,00:30,2016-08-26T00:30:00+02:00,45938.0,118.0,299.0,3048.0,38686.0,1061.0,-1.0,5580.0,966.0
28945,France,Données définitives,2016-08-26,01:00,2016-08-26T01:00:00+02:00,42874.0,118.0,314.0,2916.0,38167.0,1077.0,-1.0,4633.0,974.0
28946,France,Données définitives,2016-08-26,01:30,2016-08-26T01:30:00+02:00,42128.0,118.0,279.0,3017.0,38273.0,1077.0,-1.0,4016.0,975.0
28947,France,Données définitives,2016-08-26,02:00,2016-08-26T02:00:00+02:00,41339.0,119.0,273.0,2938.0,38105.0,1087.0,-1.0,3675.0,970.0
28948,France,Données définitives,2016-08-26,02:30,2016-08-26T02:30:00+02:00,40686.0,119.0,273.0,3030.0,37877.0,1058.0,-1.0,3669.0,985.0
28949,France,Données définitives,2016-08-26,03:00,2016-08-26T03:00:00+02:00,39139.0,118.0,278.0,3059.0,37175.0,1188.0,-1.0,3536.0,983.0
28950,France,Données définitives,2016-08-26,03:30,2016-08-26T03:30:00+02:00,38463.0,118.0,287.0,2695.0,37448.0,1368.0,-1.0,3498.0,980.0
28951,France,Données définitives,2016-08-26,04:00,2016-08-26T04:00:00+02:00,37866.0,119.0,291.0,2617.0,37268.0,1423.0,-1.0,3083.0,978.0
28952,France,Données définitives,2016-08-26,04:30,2016-08-26T04:30:00+02:00,37776.0,118.0,292.0,2624.0,37167.0,1413.0,-1.0,3030.0,980.0


In [ ]:
print(test.iloc[14477])

Date                          2016-08-26
Heure                              07:00
Consommation (MW)                43668.0
Fioul (MW)                         119.0
Charbon (MW)                       292.5
Gaz (MW)                          3524.0
Nucléaire (MW)                   38878.0
Eolien (MW)                       1195.5
Solaire (MW)                        17.0
Hydraulique (MW)                  4446.0
Bioénergies (MW)                   980.0
Datetime             2016-08-26 07:00:00
Country                           France
Datetime (Local)     2016-08-26 07:00:00
Price (EUR/MWhe)                   34.86
Series                                 0
IndexTime                          14477
Name: 14477, dtype: object


In [ ]:
print(test.iloc[63606]) #max 2987,78 on 4 April 2022 iloc=63606

Date                          2022-04-04
Heure                              08:00
Consommation (MW)                73257.0
Fioul (MW)                        1493.0
Charbon (MW)                       465.5
Gaz (MW)                          8495.5
Nucléaire (MW)                   32110.0
Eolien (MW)                       4408.5
Solaire (MW)                       720.0
Hydraulique (MW)                 16159.5
Bioénergies (MW)                  1019.5
Datetime             2022-04-04 08:00:00
Country                           France
Datetime (Local)     2022-04-04 08:00:00
Price (EUR/MWhe)                 2987.78
Series                                 0
IndexTime                          63606
Name: 63606, dtype: object
